In [33]:
import os
import numpy as np
import faiss
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer, util
from google.generativeai import GenerativeModel, configure
from dotenv import load_dotenv

In [34]:
# Load environment variables
load_dotenv()

# Configure Google Generative AI with API key
configure(api_key=os.getenv("GOOGLE_API_KEY"))
print("API Key Loaded:", os.getenv("GOOGLE_API_KEY") is not None)

# Load pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

API Key Loaded: True


In [35]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''.join(page.extract_text() for page in reader.pages)
    return text

def split_into_sections(text, section_length=500):
    """Split text into sections of approximately `section_length` words."""
    words = text.split()
    sections = [' '.join(words[i:i + section_length]) for i in range(0, len(words), section_length)]
    return sections

def embed_text(text):
    """Embed the text using the Sentence Transformer model."""
    return model.encode(text, convert_to_tensor=True)


In [36]:
def create_faiss_index(embeddings):
    """Create and return a FAISS index."""
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def query_gemini(question, history=[]):
    """Query the Gemini model and return the response."""
    model = GenerativeModel("gemini-1.5-pro")
    chat = model.start_chat(history=history)
    response = chat.send_message(question, stream=True)
    response.resolve()  # Ensure response is fully resolved
    return response, chat.history

def perform_query(query, index, sections):
    """Perform the query using FAISS search and Gemini model."""
    query_embedding = embed_text(query)
    D, I = index.search(np.array([query_embedding]), k=3)
    docs = [sections[i] for i in I[0]]
    combined_docs = " ".join(docs)
    question = f"{combined_docs}\n\n{query}"
    response, chat_history = query_gemini(question, history=[])
    response_text = "".join(chunk.text for chunk in response)
    return response_text

In [37]:
# Paths to the PDF files
pdf_path = 'jio.pdf'

# Extract and process text from the PDFs
text = extract_text_from_pdf(pdf_path)
sections = split_into_sections(text)

# Embed the text and create FAISS index
embeddings = embed_text(sections)
index = create_faiss_index(embeddings)

In [38]:
# Personal details and bot context
personal_details = {
    "name": "Rahul Sharma",
    "dob": "1990-01-15",
    "address": "123, Maple Street, Sector 45, Gurgaon, Haryana, 122003",
    "mobile": "+91 9876543210",
    "email": "rahul.sharma@example.com"
}

In [52]:
bot_context = "You are a Jio customer support staff. Answer questions very precisely. If anything asked irrelevant to jio, politely avoid the question."
query = "What are the new plans introduced in jio?"
personal_details_str = " ".join([f"{key}: {value}" for key, value in personal_details.items()])
question = f"{text}\n\n{personal_details_str}\n\n{bot_context}\n\n{query}"
# Perform the query and print the response
response_text = perform_query(query, index, sections)

In [53]:
print("Query :", query)
print("Answer :", response_text)

Query : What are the new plans introduced in jio?
Answer : The press release outlines the following changes to Jio's plans:

**Key Points:**

* **Focus on Unlimited 5G:** Jio emphasizes its continued offering of truly unlimited 5G data on its leading plans (2GB/day and above) at no extra cost.
* **Price Increases:** While the exact details are unclear for all plans, the "Tariff Table" suggests a general price increase across popular unlimited voice and SMS plans.  
* **New Plans Effective Date:** The new plans go into effect on **July 3rd, 2024**.
* **JioBharat/JioPhone Unaffected:** Tariffs for these plans remain unchanged, emphasizing affordability for feature phone users.

**New Applications (Free for a year):**

* **JioSafe:** Quantum-secure communication app (calls, messaging, file transfer) - regular price Rs 199/month.
* **JioTranslate:** AI-powered multilingual translation for voice, text, and images - regular price Rs 99/month.

**Example Tariff Changes (Limited Information):*